In [13]:
import pandas as pd
import numpy as np
import networkx as nx
import random
from scipy.spatial.distance import euclidean
from itertools import combinations
import plotly.graph_objects as go
import webbrowser
import math


In [14]:
df = pd.read_csv("tum_sentetik2.csv")
df['Kategori'] = df['Mağaza'].apply(lambda x: x.split('_')[0])

available_products = df['Ürün'].unique()
product_to_category = df.groupby('Ürün')['Kategori'].first().to_dict()

product_to_category = df.groupby('Ürün')['Kategori'].first().to_dict()

def get_shopping_list(available_products, product_to_category):
    print("\n=== Alışveriş Listesi Oluşturma ===")
    
    shopping_list = []
    print("\nAlışveriş listesine eklemek istediğiniz ürünleri girin.")
    print("Bitirmek için 'bitir' yazın.")
    
    while True:
        product = input("Ürün adı: ").strip()
        if product.lower() == 'bitir':
            break
        if product in available_products:
            shopping_list.append(product)
            print(f"{product} listeye eklendi.")
        else:
            print(f"Hata: '{product}' veri setinde bulunmuyor. Lütfen geçerli bir ürün girin.")
    
    if not shopping_list:
        raise ValueError("Alışveriş listesi boş olamaz!")
    
    print("\nOluşturulan Alışveriş Listesi:", shopping_list)
    return shopping_list

# Alışveriş listesini al
shopping_list = get_shopping_list(available_products, product_to_category)

# Ürün kategorilerini oluştur
def create_dynamic_item_categories(shopping_list, product_to_category):
    item_categories = {}
    for item in shopping_list:
        category = product_to_category.get(item, "Bilinmeyen")
        item_categories[item] = category
    return item_categories


item_categories = create_dynamic_item_categories(shopping_list, product_to_category)

# Mağaza kategorileri için renkler 
kategori_renkleri = {
    "Gida": "orange", "Kozmetik": "purple", "Giysi": "blue", "Elektronik": "red", "Ev": "green"
}

# Ürünleri içeren mağazaları bul 
filtered_stores = df[df['Ürün'].isin(shopping_list)][['Mağaza', 'X_Koordinat', 'Y_Koordinat', 'Kategori']].drop_duplicates()
valid_stores = set(filtered_stores['Mağaza'])
store_products = df[df['Ürün'].isin(shopping_list) & df['Mağaza'].isin(valid_stores)].groupby('Mağaza')['Ürün'].apply(set).to_dict()
product_prices = df[df['Ürün'].isin(shopping_list) & df['Mağaza'].isin(valid_stores)].set_index(['Mağaza', 'Ürün'])['Fiyat'].to_dict()

# Ürün-Mağaza eşleştirmesi 
item_to_stores = {item: set() for item in shopping_list}
for store in filtered_stores['Mağaza']:
    store_category = store.split('_')[0]
    for item in store_products.get(store, set()):
        if item in shopping_list and item_categories[item] == store_category:
            item_to_stores[item].add(store)


=== Alışveriş Listesi Oluşturma ===

Alışveriş listesine eklemek istediğiniz ürünleri girin.
Bitirmek için 'bitir' yazın.


Bant listeye eklendi.
Sabun listeye eklendi.
Hırka listeye eklendi.
Elbise listeye eklendi.
Bot listeye eklendi.
Oje listeye eklendi.
Tablet listeye eklendi.
Süt listeye eklendi.
Peynir listeye eklendi.

Oluşturulan Alışveriş Listesi: ['Bant', 'Sabun', 'Hırka', 'Elbise', 'Bot', 'Oje', 'Tablet', 'Süt', 'Peynir']


In [15]:
# Graph oluştur
G_sa = nx.Graph()
for _, row in filtered_stores.iterrows():
    G_sa.add_node(row['Mağaza'], pos_sa=(row['X_Koordinat'], row['Y_Koordinat']), kategori_sa=row['Kategori'])

for (m1, x1, y1), (m2, x2, y2) in combinations(filtered_stores[['Mağaza', 'X_Koordinat', 'Y_Koordinat']].values, 2):
    distance = euclidean((x1, y1), (x2, y2))
    travel_time = distance / 60
    G_sa.add_edge(m1, m2, weight=travel_time)

# Mağaza listesi
store_list = list(G_sa.nodes())

### SA FUNCITONS


In [16]:
def calculate_cost(solution, graph, product_prices, shopping_list):
    item_to_store, route = solution
    total_cost = 0
    item_sources = {}
    
    
    for item, store in item_to_store.items():
        price = product_prices.get((store, item), float('inf'))
        if price < float('inf'):
            item_sources[item] = (store, price)
            total_cost += price
        else:
            return float('inf'), item_sources
    
    
    if len(route) > 1:
        for i in range(len(route) - 1):
            store1, store2 = route[i], route[i + 1]
            if graph.has_edge(store1, store2):
                travel_cost = graph[store1][store2]['weight'] * 100  # Saat başına 100 TL
                total_cost += travel_cost
            else:
                return float('inf'), item_sources
    
    if set(item_sources.keys()) != set(shopping_list):
        return float('inf'), item_sources
    
    return total_cost, item_sources


def initial_solution(shopping_list, item_to_stores, product_prices):
 
    item_to_store = {}
    selected_stores = set()
    
    
    for item in shopping_list:
        possible_stores = item_to_stores[item]
        if not possible_stores:
            continue
        # En ucuz mağazayı bul
        best_store = min(possible_stores, key=lambda store: product_prices.get((store, item), float('inf')))
        item_to_store[item] = best_store
        selected_stores.add(best_store)
    

    for category in set(item_categories.values()):
        items_in_category = [item for item in shopping_list if item_categories[item] == category]
        if not items_in_category:
            continue
        
        category_stores = set()
        for item in items_in_category:
            category_stores.update(item_to_stores[item])
        if not category_stores:
            continue
        best_store = None
        best_cost = float('inf')
        for store in category_stores:
            cost = sum(product_prices.get((store, item), float('inf')) for item in items_in_category)
            if cost < best_cost:
                best_cost = cost
                best_store = store
        
        for item in items_in_category:
            if product_prices.get((best_store, item), float('inf')) < float('inf'):
                item_to_store[item] = best_store
    
   
    selected_stores = set(item_to_store.values())
    
    
    route = list(selected_stores)
    random.shuffle(route)
    
    return (item_to_store, route)

def get_neighbor(solution, store_list, shopping_list, item_to_stores, product_prices):
    item_to_store, route = solution
    item_to_store = item_to_store.copy()
    new_route = route.copy()
    
    
    move = random.choice(["swap_route", "replace_route", "change_store", "merge_category"])
    
    if move == "swap_route" and len(new_route) > 1:
        
        i, j = random.sample(range(len(new_route)), 2)
        new_route[i], new_route[j] = new_route[j], new_route[i]
    
    elif move == "replace_route":
        
        if len(new_route) > 1:
            idx = random.randint(0, len(new_route) - 1)
            old_store = new_route.pop(idx)
            
            selected_stores = set(item_to_store.values())
            if old_store in selected_stores:
                new_store = random.choice([s for s in selected_stores if s != old_store])
                new_route.insert(idx, new_store)
    
    elif move == "change_store":
        
        item = random.choice(list(item_to_store.keys()))
        possible_stores = list(item_to_stores[item])
        
        current_store = item_to_store[item]
        possible_stores = [s for s in possible_stores if s != current_store]
        if possible_stores:
            new_store = random.choice(possible_stores)
            item_to_store[item] = new_store
    
    elif move == "merge_category":
        
        category = random.choice(list(set(item_categories.values())))
        items_in_category = [item for item in shopping_list if item_categories[item] == category]
        if len(items_in_category) < 2:
            return (item_to_store, new_route)
        
        category_stores = set()
        for item in items_in_category:
            category_stores.update(item_to_stores[item])
        if not category_stores:
            return (item_to_store, new_route)
        best_store = None
        best_cost = float('inf')
        for store in category_stores:
            cost = sum(product_prices.get((store, item), float('inf')) for item in items_in_category)
            if cost < best_cost:
                best_cost = cost
                best_store = store
        
        for item in items_in_category:
            if product_prices.get((best_store, item), float('inf')) < float('inf'):
                item_to_store[item] = best_store
    
    
    selected_stores = set(item_to_store.values())
    new_route = [store for store in new_route if store in selected_stores]
    for store in selected_stores:
        if store not in new_route:
            new_route.append(store)
    
    return (item_to_store, new_route)


def simulated_annealing(graph, store_list, product_prices, shopping_list, item_to_stores, initial_temp=1000, cooling_rate=0.995, max_iterations=1000):
    current_solution = initial_solution(shopping_list, item_to_stores, product_prices)
    current_cost, current_sources = calculate_cost(current_solution, graph, product_prices, shopping_list)
    
    best_solution = current_solution
    best_cost, best_sources = current_cost, current_sources.copy()
    temperature = initial_temp
    
    for iteration in range(max_iterations):
        neighbor = get_neighbor(current_solution, store_list, shopping_list, item_to_stores, product_prices)
        neighbor_cost, neighbor_sources = calculate_cost(neighbor, graph, product_prices, shopping_list)
        
        cost_diff = neighbor_cost - current_cost
        if cost_diff < 0 or (neighbor_cost < float('inf') and random.random() < math.exp(-cost_diff / temperature)):
            current_solution = neighbor
            current_cost = neighbor_cost
            current_sources = neighbor_sources
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost
                best_sources = current_sources.copy()
                print(f"Iteration {iteration}: Best Cost = {best_cost:.2f}")
        
        temperature *= cooling_rate
    
    return best_solution, best_cost, sum(price for _, (store, price) in best_sources.items()), best_sources

In [17]:
# Simulated Annealing’i Çalıştır
optimal_solution, total_cost, total_price, item_sources = simulated_annealing(
    G_sa, store_list, product_prices, shopping_list, item_to_stores
)

# Sonuçları Yazdır
item_to_store, optimal_route = optimal_solution

from IPython.display import display
display(optimal_route)

print("+++++++++++++++++++++++")

print("\n=== Simulated Annealing Sonuçları ===")
print("Optimal Ürün-Mağaza Eşleştirmesi:")
for item, store in item_to_store.items():
    print(f"  {item}: {store} (Fiyat: {item_sources[item][1]:.2f} TL)")
print("\nOptimal Rota:")
for i, store in enumerate(optimal_route, 1):
    print(f"  {i}. {store}")
print(f"\nToplam Maliyet (Ulaşım + Ürün): {total_cost:.2f} TL")
print(f"Toplam Ürün Fiyatı: {total_price:.2f} TL")
missing_items = set(shopping_list) - set(item_sources.keys())
if not missing_items:
    print("\nTüm ürünler başarıyla alındı!")
else:
    print("\nEksik ürünler:", missing_items)

# Görselleştirme
pos_sa = nx.get_node_attributes(G_sa, 'pos_sa')
node_x = [pos_sa[node][0] for node in G_sa.nodes()]
node_y = [pos_sa[node][1] for node in G_sa.nodes()]
node_color = ["black" if node in optimal_route else kategori_renkleri.get(G_sa.nodes[node]['kategori_sa'], 'gray') for node in G_sa.nodes()]
node_text = [node if node in optimal_route else "" for node in G_sa.nodes()]

edge_x = []
edge_y = []
for i in range(len(optimal_route) - 1):
    x0, y0 = pos_sa[optimal_route[i]]
    x1, y1 = pos_sa[optimal_route[i + 1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

fig = go.Figure()
fig.add_trace(go.Scatter(x=edge_x, y=edge_y, line=dict(width=2, color='gray'), mode='lines', name="Rota"))
fig.add_trace(go.Scatter(
    x=node_x, y=node_y, mode='markers+text',
    marker=dict(size=10, color=node_color),
    text=node_text, textposition="top center", hoverinfo="text"
))
fig.update_layout(
    title="Optimal Alışveriş Rotası ve Ürün Seçimi (SA Algoritma)",
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)
fig.write_html("simulated_annealing.html")
webbrowser.open("simulated_annealing.html")

Iteration 0: Best Cost = 2911.41
Iteration 2: Best Cost = 2779.73
Iteration 321: Best Cost = 2777.75
Iteration 384: Best Cost = 2697.86
Iteration 392: Best Cost = 2508.20
Iteration 398: Best Cost = 2486.14
Iteration 474: Best Cost = 2471.53
Iteration 479: Best Cost = 2452.08
Iteration 645: Best Cost = 2435.95
Iteration 664: Best Cost = 2423.87
Iteration 671: Best Cost = 2383.68
Iteration 674: Best Cost = 2290.87
Iteration 724: Best Cost = 2265.27
Iteration 730: Best Cost = 2253.10
Iteration 747: Best Cost = 2082.77
Iteration 775: Best Cost = 2043.49
Iteration 794: Best Cost = 2031.49
Iteration 838: Best Cost = 2014.63
Iteration 926: Best Cost = 2005.67
Iteration 936: Best Cost = 1998.15
Iteration 963: Best Cost = 1961.53


['Elektronik_Magaza_28',
 'Giysi_Magaza_22',
 'Giysi_Magaza_0',
 'Kozmetik_Magaza_32',
 'Giysi_Magaza_22',
 'Kozmetik_Magaza_32',
 'Giysi_Magaza_22',
 'Ev_Magaza_0',
 'Gida_Magaza_19',
 'Ev_Magaza_0',
 'Ev_Magaza_15',
 'Giysi_Magaza_22',
 'Kozmetik_Magaza_32']

+++++++++++++++++++++++

=== Simulated Annealing Sonuçları ===
Optimal Ürün-Mağaza Eşleştirmesi:
  Bant: Ev_Magaza_15 (Fiyat: 146.44 TL)
  Sabun: Ev_Magaza_0 (Fiyat: 156.79 TL)
  Hırka: Giysi_Magaza_0 (Fiyat: 198.69 TL)
  Elbise: Giysi_Magaza_22 (Fiyat: 65.51 TL)
  Bot: Giysi_Magaza_22 (Fiyat: 92.61 TL)
  Oje: Kozmetik_Magaza_32 (Fiyat: 54.41 TL)
  Tablet: Elektronik_Magaza_28 (Fiyat: 493.32 TL)
  Süt: Gida_Magaza_19 (Fiyat: 67.91 TL)
  Peynir: Gida_Magaza_19 (Fiyat: 30.42 TL)

Optimal Rota:
  1. Elektronik_Magaza_28
  2. Giysi_Magaza_22
  3. Giysi_Magaza_0
  4. Kozmetik_Magaza_32
  5. Giysi_Magaza_22
  6. Kozmetik_Magaza_32
  7. Giysi_Magaza_22
  8. Ev_Magaza_0
  9. Gida_Magaza_19
  10. Ev_Magaza_0
  11. Ev_Magaza_15
  12. Giysi_Magaza_22
  13. Kozmetik_Magaza_32

Toplam Maliyet (Ulaşım + Ürün): 1961.53 TL
Toplam Ürün Fiyatı: 1306.10 TL

Tüm ürünler başarıyla alındı!


True